# Exploring JSON Files
----------------------------------

The purpose of this excercise is to get comfortable working with JSON files. Let's keep the scope small and focus on two JSON files, and if you want to go out on your own and easily scrape 500 free search engine result pages (SERP), then check out the appendix for SERP API!

# Download Data

In [ ]:
!pip install gdown==5.1.0

In [ ]:
!gdown 1ejm645rRkuIdt1VETpICmJbLGyxMCf7F
!gdown 1nGfw8MOq_gYV55uJuFQ_1wgjpSIsvVw1

Downloading...
From: https://drive.google.com/uc?id=1ejm645rRkuIdt1VETpICmJbLGyxMCf7F
To: /content/Ukraine.json
100% 33.5k/33.5k [00:00<00:00, 37.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1nGfw8MOq_gYV55uJuFQ_1wgjpSIsvVw1
To: /content/London.json
100% 30.6k/30.6k [00:00<00:00, 44.5MB/s]


# Install Spark

In [ ]:
!pip install pyspark

# Start session

We start the builder pattern `SparkSession.builder` and then chain a configuration parameter that defined the application name.

Providing a useful `appName` helps you identify which programs are running on your Spark cluster.

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .appName("Exploring_JSON_actors")\
        .getOrCreate()

# Inspect the .html and .json files

Imagine that someone just sent you the search results for these two queries. You can see that someone has been searching for 'top actors' in various cities (Kyiv, Ukraine and London, England.) The first geographic name referes to the 'top actors', and while the second set of geographic names looks duplicative, it is actually where you are telling Google to search from!

If you dig through the two JSON files, you can actually click through and see the HTML result! It is buried in the JSON file but you can find it after looking around. Just lop off the **%7C** at the end and you can see the raw webpage you grabbed.
* https://serpapi.com/searches/2667d9f1ec89c315/63f12abf33b236a2bb4e555d.html%7C

Here are the clean HTML files:
* https://serpapi.com/searches/0fb523f94d3aeeac/63f12abda3f4ef6286fc8dfd.html
* https://serpapi.com/searches/2667d9f1ec89c315/63f12abf33b236a2bb4e555d.html

You can also replace the .html and just get the .json right away, too.
* https://serpapi.com/searches/0fb523f94d3aeeac/63f12abda3f4ef6286fc8dfd.json
* https://serpapi.com/searches/2667d9f1ec89c315/63f12abf33b236a2bb4e555d.json


Now that you have examined both the .html and .json files for these two search queries, **write down three interesting observations that you see**. Pay attention to how the structure of the .html website is efficiently captured in the .json file. No code here - just your qualitative observations.

## Interesting Thing 1

**Structured Data in JSON**: One interesting observation is the structured nature of the data in the JSON files compared to the HTML files. The JSON files contain well-organized data with clear keys and values, making it easier to extract and analyze specific information programmatically. This structured format facilitates data processing and analysis tasks.

## Interesting Thing 2

**Rich Snippets:** The JSON files include rich snippets containing additional information beyond the basic search results. These snippets often provide concise summaries or key details extracted from the corresponding web pages. This additional context can be valuable for users to quickly understand the relevance of the search results without needing to visit the web pages.

## Interesting Thing 3

**Consistency and Standardization**: The JSON files exhibit a high level of consistency and standardization across different search queries. Regardless of the specific search topic, the JSON structure remains consistent, which simplifies data extraction and manipulation tasks.

# 🔴 Ukraine

I think this is the easier file to work with, so let's start here.

# Read the Ukraine .json file and printSchema()

In [ ]:
#Read the json file
df1 = spark.read.json("/content/Ukraine.json")
#getting the schema
df1.printSchema()

root
 |-- inline_images: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- link: string (nullable = true)
 |    |    |-- original: string (nullable = true)
 |    |    |-- source: string (nullable = true)
 |    |    |-- source_name: string (nullable = true)
 |    |    |-- thumbnail: string (nullable = true)
 |    |    |-- title: string (nullable = true)
 |-- organic_results: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- about_page_link: string (nullable = true)
 |    |    |-- about_page_serpapi_link: string (nullable = true)
 |    |    |-- about_this_result: struct (nullable = true)
 |    |    |    |-- source: struct (nullable = true)
 |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |-- icon: string (nullable = true)
 |    |    |    |    |-- security: string (nullable = true)
 |    |    |    |    |-- source_info_link: string (nullable = true)
 |    |    |-- cached_page_link: 

**Description**

*   This schema represents structured data from a web search engine. It contains various fields such as 'organic_results', 'pagination', 'related_searches', 'search_information', 'search_metadata', 'search_parameters', and 'serpapi_pagination', each with nested structures containing information about search results, pagination, related searches, search metadata, and search parameters.
*   Each field has specific attributes like links, titles, positions, and other relevant details, providing comprehensive information about the search results and associated metadata.



# Extract the `organic_results.snippet` from the Ukraine .json
Are there any actors listed here? What does the output look like?

In [ ]:
from pyspark.sql.functions import explode

print("Below are the snippets : \n")
#extracting and exploding the results displaying each name on a separate row
df1_a = df1.select(
    explode("organic_results.snippet").alias("Organic_results_snippet")
)
# Show the results
df1_a.show(truncate=False)

Below are the snippets : 

+---------------------------------------------------------------------------------------------------------------------------------------------------------+
|Organic_results_snippet                                                                                                                                  |
+---------------------------------------------------------------------------------------------------------------------------------------------------------+
|Alec Utgoff is a British actor best known for playing 'Alexei' in the Netflix hit show, ... Ivanna Sakhno was born on November 14, 1997 in Kiev, Ukraine.|
|From Ben Stiller to Jessica Chastain, celebrities have embraced Ukraine's president and offered support to the country's war effort.                     |
|Zelenskyy wrote on his social media page that Sean Penn is in Ukraine filming a movie about the war. He also said that Penn was in Ukraine ...           |
|Ukrainian actor Roman Matsiyta was r

**Description**

*   Yes, there are several actors listed in the output(Alec Utgoff, Roman Matsiyta ...etc.,).
*    The output displays snippets of text extracted from search results related to various actors. Each snippet provides information about the actors, such as their names, roles, activities, and interactions.




# Extract the `organic_results.snippet_highlighted_words` from the Ukraine .json
Are there any actors listed here? What does the output look like?

In [ ]:
print("Below are the snippet_highlighted_words : \n")
#Extracting the exploding snippet highlighted words which are in organic results
df1_b = df1.select(
    explode("organic_results.snippet_highlighted_words").alias("snippet_highlighted_words")
)
df1_b.show(truncate = False)

Below are the snippet_highlighted_words : 

+------------------------------+
|snippet_highlighted_words     |
+------------------------------+
|[actor best, Kiev, Ukraine]   |
|[Ukraine's]                   |
|NULL                          |
|[Ukrainian actor]             |
|[Ukrainian, actor]            |
|[actor, city, Ukrainian, Kyiv]|
|[Top Kyiv, Kyiv, Ukraine]     |
|[city, Kyiv, Ukrainian, city] |
+------------------------------+




**Description**

*   The names of the actors are not listed but there are references to actors
listed in the output.  
*   The output shows highlighted words or phrases extracted from the snippets of text related to the search results.
*   These highlighted words include terms like "actor," "Ukrainian actor," and specific locations such as "Kiev" and "Kyiv," which are relevant to actors and their activities mentioned in the search results.










# Extract the `search_parameters.q` to get the name of the search query
Does the search you extracted match the search on the HTML page?



In [ ]:
# importing col function
from pyspark.sql.functions import col
print("Below is the search query : \n")

#extracting the column and renaming it
df1_c = df1.select(col("search_parameters.q").alias("Query"))
df1_c.show(truncate=False)

Below is the search query : 

+------------------------------------+
|Query                               |
+------------------------------------+
|top actors in Kyiv,Kyiv city,Ukraine|
+------------------------------------+



**Description**

*   Yes, the search I extracted matched the search on th HTML page.



# Extract the list of 12 names listed as the first SERP result from the Ukraine .json
Where was this information hiding in the .json?

In [ ]:
print("Below is the list of 12 names listed as the first SERP result from Ukraine : \n")


df_exploded_names = df1.select(
    explode("organic_results").alias("organic_result")    #selecting and exploding array column
).select(
    explode("organic_result.rich_snippet_list").alias("List_of_Names")   #selecting a column and exploding displaying each name on a separate row
)
df_exploded_names.show(truncate=False)

Below is the list of 12 names listed as the first SERP result from Ukraine : 

+----------------+
|List_of_Names   |
+----------------+
|Ivanna Sakhno   |
|Natalie Burn    |
|Gene Stupnitsky |
|Ilia Volok      |
|Oleg Zagorodnii |
|Aleksey Gorbunov|
|Ana Layevska    |
|Larisa Polonsky |
|Anna Sten       |
|Vadim Perelman  |
|Anna Sedokova   |
|Alex Feldman    |
+----------------+



**Description**

*   The information about the list of names as the first SERP result was hiding within the "organic_results" field of the JSON structure.
*   Specifically, it was nested within the "rich_snippet_list" attribute within each "organic_result" element.



# Extract a list of the 9 websites listed on the Ukraine .json
Sometimes Google (SERP API) does not return 10 related searches, it will give you less! To be clear, I would like all of the links from the 9 positions within the organic results. Names should include imdb.com, theguardian.com, euronews.com etc.

In [ ]:
# importing required functions
from pyspark.sql.functions import explode,col

print("Below are the 9 websites that are listed : \n")

df_exploded = df1.select(explode("organic_results").alias("organic_results"))

# Select the displayed_link from each organic result
links_df = df_exploded.select("organic_results.displayed_link")

# Collecting these links as a list
links_list = links_df.collect()

# Additionally, extracting the embedded link from the first organic_result's related_results
ukraine_first_related_link = df_exploded.select(
    explode("organic_results.related_results.displayed_link").alias("related_displayed_link")
).first()

#adding the link to the websites_list
links_list.append(ukraine_first_related_link)

# Printing all the 9 links
for link in links_list:
    print(link.displayed_link if 'displayed_link' in link else link['related_displayed_link'])

Below are the 9 websites that are listed : 

https://www.imdb.com › search › name › birth_place=...
https://www.theguardian.com › world › jan › ukraine-h...
https://www.euronews.com › video › 2022/06/28 › us-act...
https://nypost.com › 2022/03/06 › ukrainian-actor-who...
https://www.newsweek.com › ... › United Nations
https://www.tribuneindia.com › Nation
https://www.tripadvisor.com › ... › Theaters in Kyiv
https://www.latimes.com › business › story › actor-sean-p...
https://www.imdb.com › search › name › birth_place=...


# 🔴 London

# Extract the list of famous actors from the first result on the London .json

```
Richard Foreman. Christian Bale. ...
Nick Briggs. Sean Bean. ...
Kate Beckinsale. ...
Dirk Bogarde. ...
Michael Caine. ...
John Cleese. ...
Sacha Baron Cohen. ..
```

This one is a bit tough to extract because it is 'hiding' in an answer box (notice how this first search result is prominent and specially formatted by Google.) It is NOT the first result (from Position 1) of the organic results.

While it is easy for you to just retrieve the answer (once you find it in the JSON), why not practice your PySpark and also clean up the information - I see 9 names in the answer box, please make me a table with one column called actors and 9 rows, one for each actor. **Hint:** You can use the `.` as a delimiter and replace all `...` with nothing...

In [ ]:
#reading the json file
london = spark.read.json("/content/London.json")
#printing the schema
london.printSchema()

root
 |-- answer_box: struct (nullable = true)
 |    |-- about_this_result: struct (nullable = true)
 |    |    |-- source: struct (nullable = true)
 |    |    |    |-- description: string (nullable = true)
 |    |    |    |-- icon: string (nullable = true)
 |    |    |    |-- security: string (nullable = true)
 |    |    |    |-- source_info_link: string (nullable = true)
 |    |-- date: string (nullable = true)
 |    |-- displayed_link: string (nullable = true)
 |    |-- images: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- link: string (nullable = true)
 |    |-- list: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- snippet: string (nullable = true)
 |    |-- snippet_highlighted_words: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- title: string (nullable = true)
 |    |-- type: string (nullable = true)
 |-- organic_results: array (nullable = true)
 |    |-- elemen

In [ ]:
from pyspark.sql.functions import regexp_replace,split
print("Below is the list of famous actors in London from the first result:")

london_1 = london.select(
     explode("answer_box.list").alias("actor")
)

#replaced ... with nothing,replaced space with nothing and exploded splitting by . and then filtered
#Explode the array of names to display each name on a separate row
london_1 = london_1.withColumn("actor", regexp_replace("actor", "\.\.\.", "")) \
                   .withColumn("actor", regexp_replace("actor", " ", "")) \
                   .withColumn("actor", explode(split("actor", "\\."))) \
                   .filter(col("actor")!= "")

london_1.show(truncate=False)

Below is the list of famous actors in London from the first result:
+---------------------------------+
|actor                            |
+---------------------------------+
|RichardForeman                   |
|ChristianBale                    |
|NickBriggs                       |
|SeanBean                         |
|KateBeckinsale                   |
|DirkBogarde                      |
|MichaelCaine                     |
|JohnCleese                       |
|SachaBaronCohen                  |
|Photograph:YANNISDRAKOULIDIS/2021|
+---------------------------------+



# Examine 'Orang juga bertanya'/'People Also Ask' in the London .json
For some reason, this London search result came from the Indonesian Google search engine. 'Orang juga bertanya' is Indonesian for 'People Also Ask'.

Click through the HTML and find out 'Who is No 1 actor in the world?', then use code to extract the answer from the .json. Print the text of the answer to get full credit.

In [ ]:
questions_exploded = london.select(
    explode("related_questions").alias("related_question")
)

# Filter for the specific question about the No 1 actor in the world
specific_question = questions_exploded.filter(
    col("related_question.question") == "Who is No 1 actor in the world?"
)

# Select the snippet from the filtered result
snippet = specific_question.select(
    "related_question.snippet"
)

# Show the snippet
snippet.show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|snippet                                                                                                                                                                                                                                                                                                          |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|American veteran actor Dwayne Johnson is the most popular actor in the worl

**Description**

*   Dwayne Johnson is the No 1 actor in the world.



# Extract a list of the 10 websites listed on the London .json
Similar to what you did for Ukraine. These come from the organic results. Should include names like timeout.com, imdb.com, etc.

In [ ]:
print("Below are the 10 websites that are listed : \n")

# Explode the organic_results to work with each result separately
london_exploded = london.select(explode("organic_results").alias("organic_result"))

# Select the displayed_link from each organic result
websites_df = london_exploded.select("organic_result.displayed_link")

# Collect these links as a list
websites_list = [row['displayed_link'] for row in websites_df.collect()]

# Extract the embedded link from the first organic_result's related_results
related_links = london_exploded.select(
    explode("organic_result.related_results.displayed_link").alias("related_displayed_link")
).collect()

# Append the first related link
websites_list.append(related_links[0]['related_displayed_link'])

# Extract the displayed_link from the answer_box
answer_box_link = london.select("answer_box.displayed_link").first()

# Append the answer box link
websites_list.append(answer_box_link['displayed_link'])

# Print the links or process them as needed
for website in websites_list:
    print(website)

Below are the 10 websites that are listed : 

https://www.imdb.com › name
https://www.mrdustbin.com › ...
https://www.glamourmagazine.co.uk › ... › Celebrity News
https://www.entoin.com › briti...
https://londranews.com › lond...
https://www.youtube.com › watch
https://www.youtube.com › w...
https://www.thegentlemansjournal.com › ...
https://www.imdb.com › list
https://www.timeout.com › film › 50-great-british-actors-t...


# 🔴 Comments


I learnt the following points by doing this assignment.


*   I learnt the complexity of handling semi-similar JSON structures, showcasing the need for versatile data extraction techniques to manage variability and nested data efficiently.
*   Using dynamic data manipulation functions like explode and select in PySpark to extract and transform data from complex JSON files
*   I also learnt how to make a list of links if a link is embedded (as in question 7 & 10) and how to append.

Furthermore, I got a good handson experience on how to deal with complex JSON files using PySpark, particularly in schema exploration and data previewing.




This one is tough because Ukraine only has 9 links and London has 10 links!

Read both .json files at once using a wildcard, and make a dataframe with two rows (one for Ukraine and one for London) and 21 columns. The first column has the search query, and the next 10 columns are the (up to) 10 websites and the 10 columns after that are the 10 snippets. Good luck!

In [ ]:
from pyspark.sql.functions import expr,row_number
from pyspark.sql.window import Window

# Read both JSON files using a wildcard
df = spark.read.json("/content/*.json")

# Extract necessary information for Ukraine
ukraine_df = df.filter(df.search_metadata.google_url.contains("Ukraine")).limit(1)
ukraine_df_links = ukraine_df.selectExpr("search_parameters.q as search_query", "explode(organic_results.link) as website").limit(10)  #extracting websites
ukraine_df_snippets = ukraine_df.selectExpr("search_parameters.q as search_query", "explode(organic_results.snippet) as snippet").limit(10)  #extracting snippets

# Extract necessary information for London
london_df = df.filter(df.search_metadata.google_url.contains("London")).limit(1)
london_df_links = london_df.selectExpr("search_parameters.q as search_query", "explode(organic_results.link) as website").limit(10)       #extracting websites
london_df_snippets = london_df.selectExpr("search_parameters.q as search_query", "explode(organic_results.snippet) as snippet").limit(10)  #extracting snippets

# Combine the link and snippet DataFrames
ukraine_combined = ukraine_df_links.join(ukraine_df_snippets, "search_query").withColumn("row_num", row_number().over(Window.partitionBy("search_query").orderBy("website"))).limit(10)
london_combined = london_df_links.join(london_df_snippets, "search_query").withColumn("row_num", row_number().over(Window.partitionBy("search_query").orderBy("website"))).limit(10)

# Pivot the combined DataFrames to get websites and snippets as columns
ukraine_pivoted = ukraine_combined.groupBy("search_query").pivot("row_num").agg(expr("first(website) as website"), expr("first(snippet) as snippet"))
london_pivoted = london_combined.groupBy("search_query").pivot("row_num").agg(expr("first(website) as website"), expr("first(snippet) as snippet"))

# Merge the DataFrames into a single DataFrame with two rows
merged_df_1 = ukraine_pivoted.union(london_pivoted)

# Show the resulting DataFrame
merged_df_1.show(truncate=False)

+---------------------------------------------------+-------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
#Finding number of rows and columns
print("Number of rows and columns :",(merged_df_1.count(),len(merged_df_1.columns)))

Number of rows and columns : (2, 21)
